In [1]:
import sys
from pathlib import Path
import os 
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[1]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import pandas as pd
import math

In [3]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import openpyxl

Excel avec 
- Les 9 files de base (6 cbm, 3 excel) 
- Un file avec la différence d'accuracy entre male et female pour chacun des groupes: en row les 9 datasets, en ordonnée "différence entre male et female du groupe 1", "du groupe 2", "total"
- 3 file, un pour balanced, imbalanced 1, imbalanced 2 pour voir les différences entre les groupes

In [8]:
"""
# R

path_dict = {
    'Baseline': 'results/imSitu/200_verbs/grid_search/baseline/lr_0.001_momentum_0.8_step_5_gamma_0.1/balanced/accuracies_by_group.csv',
    'CBM Sparse': 'results/imSitu/200_verbs_full/CBM/sparse_no_gender/accuracies_by_group.csv',
    'CBM Dense': 'results/imSitu/200_verbs_full/CBM/dense_no_gender/accuracies_by_group.csv',
    'CBM Sparse gender': 'results/imSitu/200_verbs_full/CBM/sparse_gender/accuracies_by_group.csv',
    'CBM Dense gender': 'results/imSitu/200_verbs_full/CBM/dense_gender/accuracies_by_group.csv',
}
"""

"\n# R\n\npath_dict = {\n    'Baseline': 'results/imSitu/200_verbs/grid_search/baseline/lr_0.001_momentum_0.8_step_5_gamma_0.1/balanced/accuracies_by_group.csv',\n    'CBM Sparse': 'results/imSitu/200_verbs_full/CBM/sparse_no_gender/accuracies_by_group.csv',\n    'CBM Dense': 'results/imSitu/200_verbs_full/CBM/dense_no_gender/accuracies_by_group.csv',\n    'CBM Sparse gender': 'results/imSitu/200_verbs_full/CBM/sparse_gender/accuracies_by_group.csv',\n    'CBM Dense gender': 'results/imSitu/200_verbs_full/CBM/dense_gender/accuracies_by_group.csv',\n}\n"

# 200 verbs bias experiment

In [5]:
import pandas as pd
import json

def read_txt(path):
    # Open the file and read the lines
    with open(path, 'r') as f:
        lines = f.readlines()

    # Convert each line to a dictionary and collect in a list
    data = [json.loads(line) for line in lines]

    # Convert list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)
    df['nurses_male_minus_female'] = df['nurses_male'] - df['nurses_female']
    return df


In [24]:
path_dict = {
    'Baseline Balanced': 'results/doctor_nurse/alexnet/train_balanced/accuracies.txt',
    'Baseline Male A': 'results/doctor_nurse/alexnet/train_imbalanced_1/accuracies.txt',
    'Baseline Female A': 'results/doctor_nurse/alexnet/train_imbalanced_2/accuracies.txt',
    'CBM Balanced': 'results/doctor_nurse/CBM/no_gender/balanced/accuracies.txt',
    'CBM Male A': 'results/doctor_nurse/CBM/no_gender/imbalanced_1/accuracies.txt',
    'CBM Female A': 'results/doctor_nurse/CBM/no_gender/imbalanced_2/accuracies.txt',
    'CBM Balanced Gender': 'results/doctor_nurse/CBM/gender/balanced/accuracies.txt',
    'CBM Male A Gender': 'results/doctor_nurse/CBM/gender/imbalanced_1/accuracies.txt',
    'CBM Female A Gender': 'results/doctor_nurse/CBM/gender/imbalanced_2/accuracies.txt',
}

In [25]:
result_dict = {}

for name, path in path_dict.items():
    result_dict[name] = read_txt(path)

In [32]:
result_dict['Baseline Male A']

,doctors_female,doctors_male,nurses_female,nurses_male,male accuracy,female accuracy,Doctor-Male group accuracy,Doctor-Female group accuracy,Doctors,Nurses,Total accuracy,doctors_male_minus_female,nurses_male_minus_female
0,0.833333,0.712121,0.590909,0.523077,0.617599,0.712121,0.651515,0.678205,0.772727,0.556993,0.66486,-0.121212,-0.067832


## CheatBM

In [76]:
import json
original = json.load(open('data/datasets/imSitu/data/200_verbs_full/target_concepts_retained.json'))

In [79]:
results = []

for verb, concepts in original.items():
    results += concepts

In [81]:
for i in results:
    print(i)

outdoors
paw
tightness
machine
inside
alignment
tightening
screwdriver
wrench
screw
nut
finger
garage
tie
fit
dial
knob
room
height
wheel
outside
bolt
visual aspect
picture
inside
room
miss
museum
gallery
infant
fine art
adult female
outdoors
outside
man
house
mountain
vista
palpebra
eyeshadow
brush
room
scraper
wall
eyeliner
war paint
optic
inside
nail polish
outdoors
outside
paw
finger
cream
face
lotion
applier
cheek
varnish
wood
paint
mascara
lash
lipstick
lip
paintbrush
gum
food
inside
paw
plaything
table
flower arrangement
flower
room
kitchen
outdoors
scissors
plant
paper
vase
outside
man
business office
room
meeting
classroom
teacher
educatee
adult female
outside
outdoors
somebody
room
book
miss
inside
posting
paper
man
outdoors
outside
booklet
hall
adult female
male child
photograph
building
kitchen
counter
bowl
cookie sheet
oven
batter
table
cake
inside
cooky
bread
room
dough
outdoors
pan
cupcake
tray
somebody
adult female
food
room
sidewalk
street
money
outdoors
man
outside
ro

In [9]:
results_temp = []

for name_df,df in result_dict.items():
    #if name_df in ['Baseline Imbalanced 1', 'Baseline Imbalanced 2', 'Baseline Balanced', 'Baseline full']:
    if name_df == 'Baseline':
            male_1, female_1, male_2, female_2 = df.loc['male_1', 'accuracy'], df.loc['female_1', 'accuracy'], df.loc['male_2', 'accuracy'], df.loc['female_2', 'accuracy']
    
            male_minus_female_1 = 100 * (male_1 - female_1)
            male_minus_female_2 = 100 * (male_2 - female_2)

            accuracy_parity_1 = abs(male_minus_female_1)
            accuracy_parity_2 = abs(male_minus_female_2)

            highest_accuracy_1 = 'male' if male_minus_female_1 > 0 else 'female'
            highest_accuracy_2 = 'male' if male_minus_female_2 > 0 else 'female'

            average_accuracy_1 = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'])
            average_accuracy_2 = 100 * (df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
            average_accuracy = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct'] + df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'] + df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
            
            result_temp_df = pd.DataFrame({'Experiment': [name_df], 
                              'Average Accuracy': [average_accuracy],
                              'Average Accuracy 1': [average_accuracy_1],
                              'Average Accuracy 2': [average_accuracy_2],
                              'Accuracy Parity 1': [accuracy_parity_1], 
                              'Accuracy Parity 2': [accuracy_parity_2],
                              'Highest accuracy 1': [highest_accuracy_1],
                              'Highest accuracy 2': [highest_accuracy_2],
                             })
            
            results_temp.append(result_temp_df)
            
final_temp_df = pd.concat(results_temp).set_index('Experiment')

ValueError: No objects to concatenate

In [8]:
final_temp_df

,Average Accuracy,Average Accuracy 1,Average Accuracy 2,Accuracy Parity 1,Accuracy Parity 2,Highest accuracy 1,Highest accuracy 2
Experiment,,,,,,,
Baseline,25.968007,26.738883,25.117887,1.489218,5.201586,male,female


In [9]:
results = []

for name, df in result_dict.items():
    male_1, female_1, male_2, female_2 = df.loc['male_1', 'accuracy'], df.loc['female_1', 'accuracy'], df.loc['male_2', 'accuracy'], df.loc['female_2', 'accuracy']
    
    male_minus_female_1 = 100 * (male_1 - female_1)
    male_minus_female_2 = 100 * (male_2 - female_2)
    
    accuracy_parity_1 = abs(male_minus_female_1)
    accuracy_parity_2 = abs(male_minus_female_2)
    
    highest_accuracy_1 = 'male' if male_minus_female_1 > 0 else 'female'
    highest_accuracy_2 = 'male' if male_minus_female_2 > 0 else 'female'
    
    average_accuracy_1 = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'])
    average_accuracy_2 = 100 * (df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
    average_accuracy = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct'] + df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'] + df.loc['male_2', 'count'] + df.loc['female_2', 'count'])

    percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline', 'Average Accuracy']) 
    percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline', 'Accuracy Parity 1']) 
    percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline', 'Accuracy Parity 2']) 
    
    
    result_df = pd.DataFrame({'Experiment': [name], 
                              'Average Accuracy': [average_accuracy],
                              'Average Accuracy 1': [average_accuracy_1],
                              'Average Accuracy 2': [average_accuracy_2],
                              'Accuracy Parity 1': [accuracy_parity_1], 
                              'Accuracy Parity 2': [accuracy_parity_2],
                              'Highest accuracy 1': [highest_accuracy_1],
                              'Highest accuracy 2': [highest_accuracy_2],
                              'Accuracy change to Baseline': [percentage_accuracy],
                              'Accuracy Parity 1 change to Baseline': [percentage_ap_1],
                              'Accuracy Parity 2 change to Baseline': [percentage_ap_2],
                             })
    results.append(result_df)
    
final_df = pd.concat(results).reset_index(drop=True)

In [38]:
df_balanced = final_df[final_df['Experiment'].isin(['Baseline Balanced', 'CBM Dense Balanced', 'CBM Sparse Balanced', 'CBM Dense Balanced Gender', 'CBM Sparse Balanced Gender'])]
df_imbalanced_1 = final_df[final_df['Experiment'].isin(['Baseline Imbalanced 1', 'CBM Dense Imbalanced 1', 'CBM Sparse Imbalanced 1', 'CBM Dense Imbalanced 1 Gender', 'CBM Sparse Imbalanced 1 Gender'])]
df_imbalanced_2 = final_df[final_df['Experiment'].isin(['Baseline Imbalanced 2', 'CBM Dense Imbalanced 2', 'CBM Sparse Imbalanced 2', 'CBM Dense Imbalanced 2 Gender', 'CBM Sparse Imbalanced 2 Gender'])]
df_full = final_df[final_df['Experiment'].isin(['Baseline full', 'CBM Sparse full', 'CBM Dense full', 'CBM Sparse full gender', 'CBM Dense full gender'])]

In [14]:
result_dict['Comparison Metrics'] = final_df

In [39]:
result_dict['Comparison Metrics'] = final_df
result_dict['Comparison Metrics Balanced'] = df_balanced
result_dict['Comparison Metrics Imbalanced 1'] = df_imbalanced_1
result_dict['Comparison Metrics Imbalanced 2'] = df_imbalanced_2
result_dict['Comparison Metrics full'] = df_full

In [15]:
with pd.ExcelWriter(Path.cwd() / 'results' / 'imSitu' / '200_verbs_full' / 'results_full.xlsx') as writer:
    for name, df in result_dict.items():
        df.to_excel(writer, sheet_name=name, index=False)

## Data Analysis

In [11]:
df = result_dict['Comparison Metrics']

KeyError: 'Comparison Metrics'

In [12]:
df

,accuracy,correct,count
male_1,0.326371,625.0,1915.0
male_2,0.270672,455.0,1681.0
female_1,0.330822,527.0,1593.0
female_2,0.326667,490.0,1500.0


In [15]:
metrics = ['Accuracy change to Baseline	', 'Accuracy Parity 1 change to Baseline', 'Accuracy Parity 2 change to Baseline']

balanced = ['CBM Dense Balanced', 'CBM Sparse Balanced', 'CBM Dense Balanced Gender', 'CBM Sparse Balanced Gender']
imbalanced_1 = ['CBM Dense Imbalanced 1', 'CBM Sparse Imbalanced 1', 'CBM Dense Imbalanced 1 Gender', 'CBM Sparse Imbalanced 1 Gender']
imbalanced_2 = ['CBM Dense Imbalanced 2', 'CBM Sparse Imbalanced 2', 'CBM Dense Imbalanced 2 Gender', 'CBM Sparse Imbalanced 2 Gender']

In [ ]:
result = {}


for experiment in balanced:
    result_experiment = {'Experiment': experiment}
    for metric in metrics:
        baseline_value = df.loc['Baseline Balanced', metric]
        experiment_value = df.loc[experiment, metric]
        percentage_change = (baseline_value - experiment_value) / (baseline_value)
        result_experiment[experiment] 

In [1]:
path_result = Path('results/imSitu/custom/CBM_imSitu_200_full')

path_balanced = path_result / 'imSitu_200_balanced'
path_imbalanced_1 = path_result / 'imSitu_200_imbalanced_1'
path_imbalanced_2 = path_result / 'imSitu_200_imbalanced_2'
path_male = path_result / 'imSitu_200_male'
path_female = path_result / 'imSitu_200_female'

NameError: name 'Path' is not defined